# Tutorial 4
In this tutorial, we are going to train a small Convolutional Neural Network using TensorFlow and convert it to an SNN using the few-spike encoding scheme.

The ANN and converted SNN both achieve around 99% on the MNIST test set.

## Install
Download wheel file

In [ ]:
if "google.colab" in str(get_ipython()):
    !gdown 1V_GzXUDzcFz9QDIpxAD8QNEglcSipssW
    !pip install pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
    %env CUDA_PATH=/usr/local/cuda

    !rm -rf /content/ml_genn
    !git clone https://github.com/genn-team/ml_genn.git
    !pip install ./ml_genn/ml_genn
    !pip install ./ml_genn/ml_genn_tf

Downloading...
From: https://drive.google.com/uc?id=1V_GzXUDzcFz9QDIpxAD8QNEglcSipssW
To: /content/pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
100% 8.29M/8.29M [00:00<00:00, 155MB/s]
Processing ./pygenn-5.0.0-cp310-cp310-linux_x86_64.whl
pygenn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
env: CUDA_PATH=/usr/local/cuda
Cloning into 'ml_genn'...
remote: Enumerating objects: 8253, done.
remote: Counting objects: 100% (560/560), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 8253 (delta 282), reused 300 (delta 222), pack-reused 7693
Receiving objects: 100% (8253/8253), 37.43 MiB | 20.59 MiB/s, done.
Resolving deltas: 100% (5514/5514), done.
Processing ./ml_genn/ml_genn
  Preparing metadata (setup.py) ... done
  Created wheel for ml-genn: filename=ml_genn-2.1.0-py3-none-any.whl size=109412 sha256=9431c5dd10d4023c81bef9b2c0c7ba0db404a3963875f88b1b426c2254d970c1
  Stored in directory: /tmp

## Train ANN
Firstly we define a simple ANN in Keras with two convolutional layers followed by two dense layers and train it:

In [ ]:
from tensorflow.keras import models, layers, datasets
from tensorflow.config import experimental

# Irritatingly, TF's default GPU memory allocator  allocates
# all available GPU memory - this can't be freed and would leave
# none for mlGeNN so we turn off this behaviour
for gpu in experimental.list_physical_devices("GPU"):
    experimental.set_memory_growth(gpu, True)

# Load MNIST data and normalise to [0,1]
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()
train_x = train_x.reshape((-1, 28, 28, 1)) / 255.0
test_x = test_x.reshape((-1, 28, 28, 1)) / 255.0

# Create and compile TF model
tf_model = models.Sequential([
    layers.Conv2D(16, 5, padding="valid", activation="relu", use_bias=False, input_shape=train_x.shape[1:]),
    layers.AveragePooling2D(2),
    layers.Conv2D(8, 5, padding="valid", activation="relu", use_bias=False),
    layers.AveragePooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu", use_bias=False),
    layers.Dense(64, activation="relu", use_bias=False),
    layers.Dense(train_y.max() + 1, activation="softmax", use_bias=False),
], name="simple_cnn")
tf_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fit TF model
tf_model.fit(train_x, train_y, epochs=10)

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10
1875/1875 [==============================] - 28s 14ms/step - loss: 0.2250 - accuracy: 0.9315
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0792 - accuracy: 0.9754
Epoch 3/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0576 - accuracy: 0.9826
Epoch 4/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0459 - accuracy: 0.9855
Epoch 5/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0374 - accuracy: 0.9883
Epoch 6/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0310 - accuracy: 0.9898
Epoch 7/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0275 - accuracy: 0.9910
Epoch 8/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0232 - accuracy: 0.9926
Epoch 9/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0212 - accuracy: 0.9932

## Evaluate ANN model
Now we evaluate the ANN on the MNIST test set:

In [ ]:
tf_model.evaluate(test_x, test_y)

313/313 [==============================] - 2s 4ms/step - loss: 0.0354 - accuracy: 0.9897


[0.03539454564452171, 0.9897000193595886]

## Build normalization dataset
To correctly configure the conversion algorithm, the range of activations in each layer is required. We determine this from a single, randomly selected batch of training data. Slightly awkwardly, mlGeNN takes these as an iterator so we turn them into a TF dataset:

In [ ]:
import numpy as np
from tensorflow.data import Dataset, AUTOTUNE

# ML GeNN norm dataset
norm_i = np.random.choice(train_x.shape[0], 128, replace=False)

norm_ds = Dataset.from_tensor_slices((train_x[norm_i], train_y[norm_i]))
norm_ds = norm_ds.batch(128)
norm_ds = norm_ds.prefetch(AUTOTUNE)

## Convert model
We are going to use the few-spike conversion scheme to convert the ANN to an SNN with $k=8$ timesteps per examples:

Stöckl, Christoph, and Wolfgang Maass. 2021. “Optimized Spiking Neurons Can Classify Images with High Accuracy through Temporal Coding with Two Spikes.” Nature Machine Intelligence 3(3): 230–38 ([doi](http://dx.doi.org/10.1038/s42256-021-00311-4))


In [ ]:
from ml_genn_tf.converters import FewSpike

# Build few-spike converter
converter = FewSpike(k=8, norm_data=[norm_ds])

# Convert and compile ML GeNN model
net, net_inputs, net_outputs, tf_layer_pops = converter.convert(tf_model)

## Compilation
In mlGeNN, in order to turn an abstract network description into something that can actually be used for training or inference you use a *compiler* class. Here, we ask the converter to build us a suitable compiler and specify batch size and that we don't want connectvity expanded into sparse connectivity.

In [ ]:
compiler = converter.create_compiler(prefer_in_memory_connect=False, batch_size=128)
compiled_net = compiler.compile(net, inputs=net_inputs, outputs=net_outputs)

## Evaluate SNN models
Finally, we evaluate the SNN model on the MNIST test set:

In [ ]:
with compiled_net:
    compiled_net.evaluate({net_inputs[0]: test_x},
                          {net_outputs[0]: test_y})

  0%|          | 0/84 [00:00<?, ?it/s]